In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [37]:
df=pd.read_csv('train_file.csv')
#df=df.set_index('Patient_ID')
df.head(3)

,Patient_ID,YEAR,LocationDesc,Subtopic,Greater_Risk_Question,Description,Sample_Size,Sex,Race,Grade,GeoLocation,QuestionCode,StratID1,StratID2,StratID3,StratificationType,Greater_Risk_Probability
0,1,1995,"Houston, TX",1,Ever used cocaine,"any form of cocaine, such as powder, crack, or...",335.0,Female,Hispanic or Latino,4,"(29.760427, -95.369803)",H49,1,4,0,Local,7.4793
1,2,1997,"Houston, TX",0,Currently drank alcohol,"at least one drink of alcohol, on at least 1 d...",382.0,Total,Black or African American,4,"(29.760427, -95.369803)",H42,0,3,0,Local,49.6852
2,3,1993,Nevada,0,Had their first drink of alcohol before age 13...,other than a few sips,196.0,Female,White,0,"(39.493240390000494, -117.07184056399967)",H41,1,6,2,State,47.8132


In [82]:
df

,Patient_ID,YEAR,LocationDesc,Subtopic,Greater_Risk_Question,Description,Sample_Size,Sex,Race,Grade,GeoLocation,QuestionCode,StratID1,StratID2,StratID3,StratificationType,Greater_Risk_Probability
0,1,1995,"Houston, TX",1,b'ever use cocain',b'form cocain powder crack freebas one time life',335.0,Female,Hispanic or Latino,4,"(29.760427, -95.369803)",H49,1,4,0,Local,7.4793
1,2,1997,"Houston, TX",0,b'current drank alcohol',b'least one drink alcohol least day day survey',382.0,Total,Black or African American,4,"(29.760427, -95.369803)",H42,0,3,0,Local,49.6852
2,3,1993,Nevada,0,b'first drink alcohol age year',b'sip',196.0,Female,White,0,"(39.493240390000494, -117.07184056399967)",H41,1,6,2,State,47.8132
3,4,2017,Pennsylvania,1,b'ever use cocain',b'form cocain powder crack freebas one time life',104.0,Male,Asian,4,"(40.79373015200048, -77.86070029399963)",H49,2,2,0,State,3.4554
4,5,2007,"San Francisco, CA",1,b'ever took steroid without doctor prescript',b'pill shot one time life',100.0,Female,Multiple Race,4,"(37.77493, -122.419416)",H55,1,7,0,Local,0.0000
5,6,2015,"Philadelphia, PA",1,b'ever use inhal',b'snif glue breath content aerosol spray can i...,108.0,Male,Black or African American,3,"(39.952584, -75.165222)",H50,2,3,1,Local,23.4575
6,7,2009,"San Diego, CA",0,b'current drank alcohol',b'least one drink alcohol least day day survey',230.0,Female,Total,1,"(32.715738, -117.161084)",H42,1,0,3,Local,45.7599
7,8,2007,Utah,0,b'current drank alcohol',b'least one drink alcohol least day day survey',180.0,Male,Total,1,"(39.360700171000474, -111.58713063499971)",H42,2,0,3,State,23.3038
8,9,2017,West Virginia,1,b'offer sold given illeg drug school properti',b' month survey',427.0,Total,Total,1,"(38.66551020200046, -80.71264013499967)",H58,0,0,3,State,30.7151
9,10,1999,Hawaii,1,b'ever use inhal',b'snif glue breath content aerosol spray can i...,314.0,Total,Total,1,"(21.304850435000446, -157.85774940299973)",H50,0,0,3,State,18.3384


### Taking Female and Total as 2 and Male as 1 because train data shows they are more likely to get addicted

In [38]:
l=[]
for i in df.Sex:
    if i=='Female':
        l.append(2)
    if i=='Total':
        l.append(2)
    if i=='Male':
        l.append(1)
d=pd.DataFrame()
d['Patient_ID']=df.Patient_ID
d['Sex']=l
d['Subtopic']=df.Subtopic
d['year']=df.YEAR
d['Grade']=df.Grade
d['size']=df.Sample_Size
d['label']=df.Greater_Risk_Probability

In [39]:
d.head()

,Patient_ID,Sex,Subtopic,year,Grade,size,label
0,1,2,1,1995,4,335.0,7.4793
1,2,2,0,1997,4,382.0,49.6852
2,3,2,0,1993,0,196.0,47.8132
3,4,1,1,2017,4,104.0,3.4554
4,5,2,1,2007,4,100.0,0.0000


### Cleaning and Stemming of Description and Greater_risk_question

In [40]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
stop=set(stopwords.words('english'))
sno=SnowballStemmer('english')
def clean(s):
    s=s.lower()
    f=[]
    for i in s.split():
        k=re.sub('[^A-Za-z]','',i)
        f.append(k)
    #s=b' '.join(f)
    return f
def stem(s):
    fil=[]
    for _ in s:
        if _ not in stop:
            s=(sno.stem(_).encode('utf8'))
            fil.append(s)
    s=b' '.join(fil)
    return s

In [41]:
t=[]
for i in df.Greater_Risk_Question:
    t.append(stem(clean(i)))
q=[]
for i in df.Description:
    q.append(stem(clean(i)))
df.Greater_Risk_Question=t
df.Description=q
df.head()

,Patient_ID,YEAR,LocationDesc,Subtopic,Greater_Risk_Question,Description,Sample_Size,Sex,Race,Grade,GeoLocation,QuestionCode,StratID1,StratID2,StratID3,StratificationType,Greater_Risk_Probability
0,1,1995,"Houston, TX",1,b'ever use cocain',b'form cocain powder crack freebas one time life',335.0,Female,Hispanic or Latino,4,"(29.760427, -95.369803)",H49,1,4,0,Local,7.4793
1,2,1997,"Houston, TX",0,b'current drank alcohol',b'least one drink alcohol least day day survey',382.0,Total,Black or African American,4,"(29.760427, -95.369803)",H42,0,3,0,Local,49.6852
2,3,1993,Nevada,0,b'first drink alcohol age year',b'sip',196.0,Female,White,0,"(39.493240390000494, -117.07184056399967)",H41,1,6,2,State,47.8132
3,4,2017,Pennsylvania,1,b'ever use cocain',b'form cocain powder crack freebas one time life',104.0,Male,Asian,4,"(40.79373015200048, -77.86070029399963)",H49,2,2,0,State,3.4554
4,5,2007,"San Francisco, CA",1,b'ever took steroid without doctor prescript',b'pill shot one time life',100.0,Female,Multiple Race,4,"(37.77493, -122.419416)",H55,1,7,0,Local,0.0000


### From here I am going to convert Description and Greater_question into vectors and then train and run my model on them to get their mean sq error value. I will use this value on the final model as a feature. why I did this? Because both of them are text so we have to convert them into numbers.

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
dff=TfidfVectorizer(lowercase=False)
X=dff.fit_transform(df.Description)
y=df.Greater_Risk_Probability

In [43]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
dl=LinearRegression()
dl.fit(X,y)
des=dl.predict(X)
from sklearn import metrics
print('MSE:', metrics.mean_squared_error(y, des))

MSE: 74.01179138932851


In [44]:
gf=TfidfVectorizer(lowercase=False)
X=gf.fit_transform(df.Greater_Risk_Question)
y=df.Greater_Risk_Probability
gl=LinearRegression()
gl.fit(X,y)
gre=gl.predict(X)
print('MSE:', metrics.mean_squared_error(y, gre))

MSE: 72.7774433927208


### This is our final table.

In [45]:
d['des']=des
d['gre']=gre
d.head()

,Patient_ID,Sex,Subtopic,year,Grade,size,label,des,gre
0,1,2,1,1995,4,335.0,7.4793,10.822977,10.822977
1,2,2,0,1997,4,382.0,49.6852,48.866800,47.946214
2,3,2,0,1993,0,196.0,47.8132,29.903339,29.903339
3,4,1,1,2017,4,104.0,3.4554,10.822977,10.822977
4,5,2,1,2007,4,100.0,0.0000,7.531894,7.531894


In [66]:
from sklearn.model_selection import train_test_split
X=d.drop('label',axis=1)
y=d.label
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=.33)

In [67]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
lf=LinearRegression()
lf.fit(x_train,y_train)
predictions=lf.predict(x_test)
from sklearn import metrics
#print("acc=",accuracy_score(y_test,predictions))
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

MAE: 5.815061548194596
MSE: 63.68037744710956
RMSE: 7.979998586911501


### Reading Test data

In [48]:
test=pd.read_csv('test_file.csv')
test.head(3)

,Patient_ID,YEAR,LocationDesc,Subtopic,Greater_Risk_Question,Description,Sample_Size,Sex,Race,Grade,GeoLocation,QuestionCode,StratID1,StratID2,StratID3,StratificationType
0,55400,2007,Navajo,1,Ever used marijuana,"also called grass, pot, or weed, one or more t...",157.0,Male,Hispanic or Latino,3,NaN,H46,2,4,1,Other
1,55401,2001,Florida,1,Currently used marijuana,"also called grass, pot, or weed, one or more t...",724.0,Total,Total,2,"(28.932040377000476, -81.92896053899966)",H48,0,0,4,State
2,55402,2005,"Charlotte-Mecklenburg County, NC",1,"Were offered, sold, or given an illegal drug o...",during the 12 months before the survey,105.0,Male,Hispanic or Latino,4,"(35.227087, -80.843127)",H58,2,4,0,Local


### Same procedure i.e converting gender and Cleaning and stemming

In [49]:
l=[]
for i in test.Sex:
    if i=='Female':
        l.append(2)
    if i=='Total':
        l.append(2)
    if i=='Male':
        l.append(1)
t=pd.DataFrame()
t['Patient_ID']=test.Patient_ID
t['Sex']=l
t['Subtopic']=test.Subtopic
t['year']=test.YEAR
t['Grade']=test.Grade
t['size']=test.Sample_Size
t.head(3)

,Patient_ID,Sex,Subtopic,year,Grade,size
0,55400,1,1,2007,3,157.0
1,55401,2,1,2001,2,724.0
2,55402,1,1,2005,4,105.0


In [50]:
p=[]
for i in test.Greater_Risk_Question:
    p.append(stem(clean(i)))
q=[]
for i in test.Description:
    q.append(stem(clean(i)))
t['gre']=p
t['des']=q
t.head()

,Patient_ID,Sex,Subtopic,year,Grade,size,gre,des
0,55400,1,1,2007,3,157.0,b'ever use marijuana',b'also call grass pot weed one time life'
1,55401,2,1,2001,2,724.0,b'current use marijuana',b'also call grass pot weed one time day survey'
2,55402,1,1,2005,4,105.0,b'offer sold given illeg drug school properti',b' month survey'
3,55403,1,1,2017,2,157.0,b'ever use synthet marijuana',b'also call k spice fake weed king kong yucata...
4,55404,2,1,2007,1,174.0,b'ever use heroin',b'also call smack junk china white one time life'


### Final test table before applying last model

In [54]:
X=gf.transform(t.gre)
gre=gl.predict(X)
x=dff.transform(t.des)
des=dl.predict(x)
t.gre=gre
t.des=des
t.head(3)

,Patient_ID,Sex,Subtopic,year,Grade,size,gre,des
0,55400,1,1,2007,3,157.0,47.976255,48.301865
1,55401,2,1,2001,2,724.0,24.173984,24.082021
2,55402,1,1,2005,4,105.0,33.056182,33.056182


In [68]:
predictions=lf.predict(t)
predictions

array([51.10632928, 24.59496277, 35.15505341, ..., 18.14677688,
       12.26103732, 64.74072971])

In [70]:
for i in range(0,len(predictions)):
    predictions[i]=round(predictions[i],4)
predictions

array([51.1063, 24.595 , 35.1551, ..., 18.1468, 12.261 , 64.7407])

In [78]:
pd.read_csv('sample_submission08f968d49116f6.csv')

,Patient_ID,Greater_Risk_Probability
0,212114,19.9652
1,212115,70.4790
2,212116,22.5626
3,212117,37.4393
4,212118,35.3642


### Submission table

In [79]:
sav=pd.DataFrame()
sav['Patient_ID']=test.Patient_ID
sav['Greater_Risk_Probability']=predictions
sav.head(2)

,Patient_ID,Greater_Risk_Probability
0,55400,51.1063
1,55401,24.5950


In [80]:
sav.to_csv('sub.csv',index=False)